## CMI Detect Sleep States - Infer

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cmi-deep-sleep-states-train/model_best.pth
/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet
/kaggle/input/child-mind-institute-detect-sleep-states/sample_submission.csv
/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv
/kaggle/input/child-mind-institute-detect-sleep-states/test_series.parquet


### Configuration

In [2]:
import gc
import time
import json
from datetime import datetime
import os
import joblib
import random
import math
from math import pi, sqrt, exp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from scipy.interpolate import interp1d
import sklearn, sklearn.model_selection
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.metrics import average_precision_score
from timm.scheduler import CosineLRScheduler
plt.style.use("ggplot")

from pyarrow.parquet import ParquetFile
import pyarrow as pa
import ctypes

torch.set_num_interop_threads(4)
torch.set_num_threads(4)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class PATHS:
    MAIN_DIR = "/kaggle/input/child-mind-institute-detect-sleep-states/"
    # csv files :
    SUBMISSION = MAIN_DIR + "sample_submission.csv"
    TRAIN_EVENTS = MAIN_DIR + "train_events.csv"
    
    # parquet files:
    TRAIN_SERIES = MAIN_DIR + "train_series.parquet"
    TEST_SERIES = MAIN_DIR + "test_series.parquet"


### Data

In [4]:
class DataReader:
    def __init__(self, demo_mode):
        super().__init__()
        
        # mapping for data loading
        self.names_mapping = {
            "submission": {
                "path": PATHS.SUBMISSION,
                "is_parquet": False,
                "has_timestamp": False,
            },
            "train_events": {
                "path": PATHS.TRAIN_EVENTS,
                "is_parquet": False,
                "has_timestamp": True,
            },
            "train_series": {
                "path": PATHS.TRAIN_SERIES,
                "is_parquet": True,
                "has_timestamp": True,
            },
            "test_series": {
                "path": PATHS.TEST_SERIES,
                "is_parquet": True,
                "has_timestamp": True,
            },
        }
        
        self.valid_names = [
            "submission",
            "train_events",
            "train_series",
            "test_series",
        ]
        
        self.demo_mode = demo_mode
    
    def verify(self, data_name):
        """変数名のバリデーション"""
        if data_name not in self.valid_names:
            print("PLEASE ENTER A VALID DATASET NAME, VALID NAMES ARE: ", valid_names)
        
        return
    
    def cleaning(self, data):
        """drop na values"""
        before_cleaning = len(data)
        print("Number of missing timestamps: ", len(data[data["timestamp"].isna()]))
        
        data = data.dropna(subset=["timestamp"])
        after_cleaning = len(data)
        
        print(
            "Percentage of removed rows: {:.1f}%".format(
                100*(before_cleaning - after_cleaning) / before_cleaning
            )
        )
        
        return data
    
    @staticmethod
    def reduce_memory_usage(data):
        """iterate through all the columns of a dataframe and modify the data type to reduce memory usage"""
        start_mem = data.memory_usage().sum() / 1024**2
        print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
        
        for col in data.columns:
            col_type = data[col].dtype
            if col_type != object:
                c_min = data[col].min()
                c_max = data[col].max()
                
                # 型によってメモリサイズを最適化
                if str(col_type)[:3] == "int":
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        data[col] = data[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        data[col] = data[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        data[col] = data[col].astype(np.int32)
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        data[col] = data[col].astype(np.int64)
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        data[col] = data[col].astype(np.float16)
                    elif c_min > np.iinfo(np.float32).min and c_max < np.iinfo(np.float32).max:
                        data[col] = data[col].astype(np.float32)
                    else:
                        data[col] = data[col].astype(np.float64)
            else:
                data[col] = data[col].astype("category")
        
        end_mem = data.memory_usage().sum() / 1024**2
        
        print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
        print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
        
        return data
    
    def load_data(self, data_name):
        """データのロード"""
        self.verify(data_name)
        data_props = self.names_mapping[data_name]
        
        if data_props["is_parquet"]:
            if self.demo_mode:
                pf = ParquetFile(data_props["path"])
                demo_rows = next(pf.iter_batches(batch_size=20_000))
                data = pa.Table.from_batches([demo_rows]).to_pandas()
            else:
                data = pd.read_parquet(data_props["path"])
        else:
            if self.demo_mode:
                data = pd.read_csv(data_props["path"], nrows=20_000)
            else:
                data = pd.read_csv(data_props["path"])
        
        gc.collect()
        
        if data_props["has_timestamp"]:
            print("cleaning")
            data = self.cleaning(data)
            
            gc.collect()
        
        data = self.reduce_memory_usage(data)
        
        return data

In [5]:
reader = DataReader(demo_mode=False)
test_series = reader.load_data(data_name="test_series")
ids = test_series.series_id.unique()

gc.collect()

cleaning
Number of missing timestamps:  0
Percentage of removed rows: 0.0%
Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.02 MB
Decreased by -92.2%


0

In [6]:
SIGMA = 720
SAMPLE_FREQ = 12

class SleepDataset(Dataset):
    def __init__(self, series_ids, series):
        series = series.reset_index()
        self.data = []
        
        for viz_id in tqdm(series_ids):
            self.data.append(series.loc[(series.series_id==viz_id)].copy().reset_index())
    
    def downsample_seq_generate_features(self, feat, downsample_factor=SAMPLE_FREQ):
        if len(feat) % SAMPLE_FREQ != 0:
            feat = np.concatenate([feat, np.zeros(SAMPLE_FREQ - len(feat) % SAMPLE_FREQ)+feat[-1]])
        
        # generate seq
        feat = np.reshape(feat, (-1, SAMPLE_FREQ))
        feat_mean = np.mean(feat, 1)
        feat_std = np.std(feat, 1)
        feat_median = np.median(feat, 1)
        feat_max = np.max(feat, 1)
        feat_min = np.min(feat, 1)
        
        return np.dstack([feat_mean, feat_std, feat_median, feat_max, feat_min])[0]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = self.data[idx][["anglez", "enmo"]].values.astype(np.float32)
        
        X = np.concatenate(
            [
                self.downsample_seq_generate_features(
                    X[:,i],
                    SAMPLE_FREQ,
                ) for i in range(X.shape[1])
            ],
            -1,
        )
        
        X = torch.from_numpy(X)
        
        return X
    
test_ds = SleepDataset(test_series.series_id.unique(), test_series)

del test_series
gc.collect()

  0%|          | 0/3 [00:00<?, ?it/s]

22

### Model

In [7]:
class ResidualBiGRU(nn.Module):
    """
    残差接続を行う双方向GRU
    """
    def __init__(self, hidden_size, n_layers=1, bidir=True):
        super().__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        dir_factor = 2 if bidir else 1
        
        # @see: https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
        self.gru = nn.GRU(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=n_layers,
            bias=True, # use bias weights
            batch_first=True, # (seq, batch,feature)を(batch, seq, feature)へ
            dropout=0,
            bidirectional=True,
        )
        self.fc1 = nn.Linear(
            in_features=hidden_size*dir_factor,
            out_features=hidden_size*dir_factor*2,
        )
        self.ln1 = nn.LayerNorm(normalized_shape=hidden_size*dir_factor*2)
        self.fc2 = nn.Linear(
            in_features=hidden_size*dir_factor*2,
            out_features=hidden_size,
        )
        self.ln2 = nn.LayerNorm(normalized_shape=hidden_size)
        
    def forward(self, x, h=None):
        """順伝播"""
        output, new_h = self.gru(x, h)
        
        output = self.fc1(output)
        output = self.ln1(output)
        output = F.relu(output)
        
        output = self.fc2(output)
        output = self.ln2(output)
        output = F.relu(output)
        
        # skip connection
        output = output + x
        
        return output, new_h
        
class MultiResidualBiGRU(nn.Module):
    """
    ResidulaBiGRUをn_layers分だけ重ねる
    """
    def __init__(self, input_size, hidden_size, out_size, n_layers, bidir=True):
        super().__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_size = out_size
        self.n_layers = n_layers
        
        self.fc_in = nn.Linear(
            in_features=input_size,
            out_features=hidden_size,
        )
        self.ln = nn.LayerNorm(normalized_shape=hidden_size)
        self.bigrus = nn.ModuleList(
            [
                ResidualBiGRU(
                    hidden_size=hidden_size,
                    n_layers=1,
                    bidir=bidir,
                )
                for _ in range(n_layers)
            ]
        )
        self.fc_out = nn.Linear(
            in_features=hidden_size,
            out_features=out_size,
        )
        
    def forward(self, x, h=None):
        """順伝播"""
        if h is None:
            # (re)initialize the hidden state
            h = [None for _ in range(self.n_layers)]
            
        x = self.fc_in(x)
        x = self.ln(x)
        x = F.relu(x)
        
        new_h = []
        for i, bigru in enumerate(self.bigrus):
            x, new_hi = bigru(x, h[i])
            new_h.append(new_hi)
            
        x = self.fc_out(x)
        
        return x, new_h # log probabilities + hidden states

### Predict

In [8]:
max_chunk_size = 24*60*100
min_interval = 30

In [9]:
model = MultiResidualBiGRU(
    input_size=10,
    hidden_size=64,
    out_size=2,
    n_layers=5,
).to(device)

model.load_state_dict(
    torch.load(
        "/kaggle/input/cmi-deep-sleep-states-train/model_best.pth",
        map_location=device,
    )
)

model.eval()
submission = pd.DataFrame()
for i in range(len(test_ds)):
    X = test_ds[i].half().to(device)
    
    seq_len = X.shape[0]
    h = None
    pred = torch.zeros((len(X), 2)).half()
    for j in range(0, seq_len, max_chunk_size):
        y_pred, h = model(X[j: j+max_chunk_size].float(), h)
        h = [hi.detach() for hi in h]
        pred[j: j+max_chunk_size] = y_pred.detach()
        
        del y_pred
        gc.collect()
        
    del h, X
    gc.collect()
    
    pred = pred.numpy()
    
    series_id = ids[i]
    days = len(pred)/(17280/12)
    scores0 = np.zeros(len(pred), dtype=np.float16)
    scores1 = np.zeros(len(pred), dtype=np.float16)
    
    for idx in range(len(pred)):
        if pred[idx, 0] == max(pred[max(0, idx-min_interval): idx+min_interval, 0]):
            scores0[idx] = max(pred[max(0, idx-min_interval): idx+min_interval, 0])
        if pred[idx, 1] == max(pred[max(0, idx-min_interval): idx+min_interval, 1]):
            scores1[idx] = max(pred[max(0, idx-min_interval): idx+min_interval, 1])
            
    candidates_onset = np.argsort(scores0)[-max(1, round(days)):]
    candidates_wakeup = np.argsort(scores1)[-max(1, round(days)):]
    
    # onset
    onset = test_ds.data[i][["step"]].iloc[
        np.clip(
            a=candidates_onset*12,
            a_min=0,
            a_max=len(test_ds.data[i])-1,
        )
    ].astype(np.int32)
    onset["event"] = "onset"
    onset["series_id"] = series_id
    onset["score"] = scores0[candidates_onset]
    
    # wakeup
    wakeup = test_ds.data[i][["step"]].iloc[
        np.clip(
            a=candidates_wakeup*12,
            a_min=0,
            a_max=len(test_ds.data[i])-1,
        )
    ].astype(np.int32)
    wakeup["event"] = "wakeup"
    wakeup["series_id"] = series_id
    wakeup["score"] = scores1[candidates_wakeup]
    
    submission = pd.concat([submission, onset, wakeup], axis=0)
    
    del onset, wakeup, candidates_onset, candidates_wakeup, scores0, scores1, pred, series_id
    gc.collect()
    
submission = submission.sort_values(["series_id", "step"]).reset_index(drop=True)
submission["row_id"] = submission.index.astype(int)
submission["score"] = submission["score"].fillna(submission["score"].mean())

submission = submission[["row_id", "series_id", "step", "event", "score"]]

submission.to_csv("submission.csv", index=False)

In [10]:
submission

,row_id,series_id,step,event,score
0,0,038441c925bb,0,onset,0.137085
1,1,038441c925bb,144,wakeup,0.000000
2,2,03d92c9f6f8a,24,onset,0.188477
3,3,03d92c9f6f8a,144,wakeup,0.000000
4,4,0402a003dae9,48,wakeup,0.003283
5,5,0402a003dae9,144,onset,0.438232
